# Implementing k-means clustering for Wikipedia Article Segmentation

##### This is based on assignment in the Machine Learning specialization by University of Washington on Coursera

In [86]:
import numpy as ny 
import pandas as pd
import math 
import operator
import random
import warnings
warnings.filterwarnings('ignore')

In [85]:
people_wiki = pd.read_csv("people_wiki.csv")

In [88]:
data_km=people_wiki.head(50)

In [89]:
text=data_km['text']  ## to get the cell value from a dataframe

In [90]:
word_count= []
for i in range(0,len(text)):
    text_words=text[i].split()
    word_list = {}
    for words in text_words:
        if words in word_list:
            word_list[words]=word_list[words]+1
        else:
            word_list[words]=1
    word_count.append(word_list)

In [91]:
data_km['word_count']=word_count

In [93]:
words_idf = {} 
for names in data_km['name']:
    curr=data_km[data_km['name']==names]
    for words in curr.iloc[0]['word_count'].keys():
        for i in range(0,len(data_km)):
            if words in data_km.iloc[i]['word_count'].keys():
                if words not in words_idf.keys():
                    words_idf[words]=1
                else:
                    words_idf[words]=words_idf[words]+1       

In [94]:
tf_idf_list=[]
for names in data_km['name']:
    tf_idf={}
    curr=data_km[data_km['name']==names]
    for words in curr.iloc[0]['word_count'].keys():
        tf_idf[words]=math.log((curr.iloc[0]['word_count'][words])/(1+words_idf[words]))
    tf_idf_list.append(tf_idf)
data_km['tf_idf']=tf_idf_list

In [95]:
# list of all the word that appear in the choosen articles
text1=data_km['text']  ## to get the cell value from a dataframe
word_list1= []
for i in range(0,len(text1)):
    text_words=text1[i].split()
    for words in text_words:
        if words not in word_list1:
            word_list1.append(words)

In [106]:
# stting the intial centroids
centroid=[]

In [102]:
min_size=math.inf
for names in data_km['name']:
    curr=data_km[data_km['name']==names]
    for words in curr.iloc[0]['tf_idf'].keys():
        if curr.iloc[0]['tf_idf'][words] < min_size:
            min_size=curr.iloc[0]['tf_idf'][words]
max_size=-math.inf
for names in data_km['name']:
    curr=data_km[data_km['name']==names]
    for words in curr.iloc[0]['tf_idf'].keys():
        if curr.iloc[0]['tf_idf'][words] > max_size:
            max_size=curr.iloc[0]['tf_idf'][words]

In [107]:
for i in range(0,6):
    temp_list = {}
    for words in word_list1:
        temp_list[words]=random.uniform(min_size,max_size)
    centroid.append(temp_list)

In [108]:
data_km['labels']=-1

In [ ]:
for a in range(0,3):    
    for names in data_km['name']:
            curr=data_km[data_km['name']==names]
            zi=-1
            dist_old=math.inf
            for i in range(0,6):       
                dist=0
                for words in curr.iloc[0]['tf_idf'].keys():
                    if words in centroid[i].keys():
                        dist=dist+(centroid[i][words]-curr.iloc[0]['tf_idf'][words])*(centroid[i][words]-curr.iloc[0]['tf_idf'][words])
                if dist < dist_old:
                    dist_old=dist
                    zi=i
            data_km['labels'][data_km['name']==names]=zi 

    print('Count 0:', len(data_km[data_km['labels']==0]),'Count 1:', len(data_km[data_km['labels']==1]),'Count 2:', len(data_km[data_km['labels']==2]),'Count 3:', len(data_km[data_km['labels']==3]),'Count 4:', len(data_km[data_km['labels']==4]), 'Count 5:', len(data_km[data_km['labels']==5]))

    for i in range(0,6):
        for words in centroid[i].keys():
            sum=0
            for names in data_km['name'][data_km['labels']==i]:
                curr=data_km[data_km['name']==names] 
                if words in curr.iloc[0]['tf_idf'].keys():
                    sum=sum+curr.iloc[0]['tf_idf'][words]
            centroid[i][words]=sum/len(data_km['name'][data_km['labels']==i]) 